In [1]:
import pandas as pd
import numpy as np 
import os
import shutil
import torch

In [2]:
Base_path_to_datas_to_train = r"C:\Users\Asus\Documents\Ship Classification\flask\input\train"
Base_path_to_datas_to_val = r"C:\Users\Asus\Documents\Ship Classification\flask\input\valid"
Base_path_to_datas_to_test = r"C:\Users\Asus\Documents\Ship Classification\flask\input\test"


PIN_MEMORY=False
#=======================================================================
clas_of_img={}
#=======================================================================
results_dict = {}
modeli=[]
#=======================================================================
torch.manual_seed(5017)
torch.cuda.manual_seed(5017)

In [3]:
def gen_classes(path,class_dict):
    for indx, path in enumerate(os.listdir(Base_path_to_datas_to_train)):
        class_dict[indx] = path
        
        
    return len(class_dict)
        

In [4]:
Num_classes = gen_classes(Base_path_to_datas_to_train,clas_of_img)
clas_of_img

{0: 'Aircraft Carrier',
 1: 'Bulkers',
 2: 'Car Carrier',
 3: 'Container Ship',
 4: 'Cruise',
 5: 'DDG',
 6: 'Recreational',
 7: 'Sailboat',
 8: 'Submarine',
 9: 'Tug'}

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator# type: ignore
from keras.applications.vgg16 import preprocess_input# type: ignore
from tensorflow.keras.applications import VGG16# type: ignore
     
train_datagen = ImageDataGenerator(
    rotation_range=45,
    horizontal_flip=True,
    width_shift_range=0.5,
    height_shift_range=0.5,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [6]:
train_set = train_datagen.flow_from_directory(Base_path_to_datas_to_train, batch_size=32, target_size=(224,224))


Found 8536 images belonging to 10 classes.


In [7]:
validation_set = train_datagen.flow_from_directory(Base_path_to_datas_to_val, batch_size=32,target_size=(224,224),shuffle=False)


Found 942 images belonging to 10 classes.


In [8]:
test_set = test_datagen.flow_from_directory(Base_path_to_datas_to_test, batch_size=32,target_size=(224,224))

Found 781 images belonging to 10 classes.


In [9]:
from keras.layers import Dense,Flatten,Dropout
from keras.models import Model

In [28]:
def create_model(input_shape,n_classes,optimizer='rmsprop'):
    conv_base=VGG16(include_top=False,weights='imagenet',input_shape=input_shape)
    for layer in conv_base.layers:
        layer.trainable = False
    top_model=conv_base.output
    top_model=Flatten(name="flatten")(top_model)
    top_model=Dense(700,activation='relu')(top_model)
    top_model=Dense(1272,activation='relu')(top_model)
    top_model=Dropout(0,2)(top_model)
    output_layer=Dense(n_classes, activation='softmax')(top_model)
    model=Model(inputs=conv_base.input,outputs=output_layer)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [29]:
from tensorflow.keras.optimizers import SGD# type: ignore

In [30]:
input_shape=(224,224,3)
optim=SGD(learning_rate=0.001)
n_classes=10
vgg_model=create_model(input_shape,n_classes,optim)

In [31]:
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [32]:
from keras.callbacks import ModelCheckpoint
cp=ModelCheckpoint('best1.h5',monitor='val_loss',verbose=1,save_best_only=True)

In [33]:
epoch=18
history=vgg_model.fit_generator(generator=train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                validation_steps=validation_set.n//validation_set.batch_size,
                                validation_data=validation_set,
                                callbacks=[cp],
                                epochs=epoch)

C:\Users\Asus\AppData\Local\Temp\ipykernel_28976\3194689763.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=vgg_model.fit_generator(generator=train_set,


Epoch 1/18


266/266 [==============================] - ETA: 0s - loss: 1.9115 - accuracy: 0.5829
Epoch 1: val_loss improved from inf to 1.35399, saving model to best1.h5


c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


266/266 [==============================] - 504s 2s/step - loss: 1.9115 - accuracy: 0.5829 - val_loss: 1.3540 - val_accuracy: 0.6455
Epoch 2/18
266/266 [==============================] - ETA: 0s - loss: 0.9181 - accuracy: 0.7437
Epoch 2: val_loss improved from 1.35399 to 1.11966, saving model to best1.h5
266/266 [==============================] - 493s 2s/step - loss: 0.9181 - accuracy: 0.7437 - val_loss: 1.1197 - val_accuracy: 0.7284
Epoch 3/18
266/266 [==============================] - ETA: 0s - loss: 0.7535 - accuracy: 0.7816
Epoch 3: val_loss improved from 1.11966 to 0.98739, saving model to best1.h5
266/266 [==============================] - 497s 2s/step - loss: 0.7535 - accuracy: 0.7816 - val_loss: 0.9874 - val_accuracy: 0.7532
Epoch 4/18
266/266 [==============================] - ETA: 0s - loss: 0.6609 - accuracy: 0.8055
Epoch 4: val_loss improved from 0.98739 to 0.93182, saving model to best1.h5
266/266 [==============================] - 497s 2s/step - loss: 0.6609 - accuracy: 0.

In [54]:
vgg_model.save('vgg16-ship-calssification.h5')

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
